# data for table 

In [2]:
import pandas as pd

In [3]:
prozorro = pd.read_json('data_short_line.json', dtype={'hospital_edrpou': 'str'})
# prozorro['id_item_short'] = 'type:' + prozorro.id_item.str[:3]
prozorro['id_item_short'] = ["type:"+a[:2]if a[:2] != '33' else "type:"+a[:3] for a in prozorro.id_item]
prozorro.date = pd.to_datetime(prozorro.date, utc=True)
prozorro['date'] = prozorro.date.dt.tz_localize(None)
prozorro = prozorro[~prozorro.date.isnull()]
prozorro['date_month'] = pd.to_datetime(prozorro.date.map(lambda x: str(x.year) + "-" + str(x.month)), format="%Y-%m")

prozorro = pd.merge(prozorro, prozorro.groupby('hospital_name', as_index=False)['sum'].size()\
         .to_frame('number_of_tenders'), how='left', on='hospital_name')
prozorro = prozorro.loc[prozorro.hospital_edrpou != '05766356']


In [4]:
top_100_per_category = prozorro.groupby('id_item_short').apply(lambda x: x.nlargest(100, 'sum')).reset_index(drop=True)
(top_100_per_category[['hospital_name', 'overal_title', 'id_item_short', 'sum']]
     .to_json('top_100_per_category.json', 'records', force_ascii=False))

In [5]:
hospitals_grouped_by_regions = pd.read_csv('hospitals_grouped_by_regions.csv')


In [13]:
table_data = pd.merge(top_100_per_category, hospitals_grouped_by_regions,
         left_on='hospital_edrpou', right_on='le_transfer',
         how='left'
        )

In [24]:
table_data[~table_data.oblast_name.isnull()].columns

Index(['date', 'hospital_edrpou', 'hospital_name', 'id', 'id_item',
       'item_description', 'overal_title', 'quantity', 'scheme', 'sum', 'unit',
       'unit_code', 'id_item_short', 'date_month', 'number_of_tenders',
       'le_transfer', 'name', 'oblast_name'],
      dtype='object')

In [45]:
output = (
    table_data[~table_data.oblast_name.isnull()]
         [[
           'hospital_name', 'overal_title',
           'hospital_edrpou','id_item_short',
           'oblast_name', 'sum'
         ]]
     )
output.loc[output.oblast_name.str.contains('м. Київ'), 'oblast_name'] = 'Київ'

output.to_json('top_100_per_category.json', 'records', force_ascii=False)

# data for slope charts

# data for doctors table

In [1]:
import pandas as pd
declarations = pd.read_excel('Декларації_в_законтрактованих_закладах_11_04_19.xlsx', dtype={'le_edrpou': 'str', 
                                                                                           'le_transfer': 'str'})

In [2]:
declarations['fop'] = (declarations.owner_proprety_type == 'ФОП').astype(int)

In [3]:
columns_needed = ['decl_count','division_settlement', 'division_address',
                  'le_transfer', 'le_name','fop',
                  'doctor_full_name', 
                  'da_area']

In [4]:
declarations.loc[declarations.owner_proprety_type == 'ФОП', 'le_transfer'] = declarations.loc[declarations.owner_proprety_type == 'ФОП', 'le_name']
declarations_short = declarations[columns_needed]

In [5]:
declarations_short['total_decl_count'] = (declarations_short
                                          .groupby(['doctor_full_name', 'da_area'])['decl_count']
                                          .transform('sum'))

/home/ptrbdr/git/medical_reform/env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
declarations_short_unique = (declarations_short
                            .drop_duplicates(['doctor_full_name', 'da_area'])
                            .drop(['decl_count'],axis=1)
                            .reset_index(drop=True))

In [7]:
declarations_short_unique['division_address'] = (declarations_short_unique.division_address.str.split(', ')
                                                 .str[-2:].str.join(', '))


In [8]:
declarations_short_unique['division_settlement'] = declarations_short_unique.division_settlement.str.title()
declarations_short_unique['le_name'] = declarations_short_unique.le_name.str.title()
declarations_short_unique['da_area'] = declarations_short_unique.da_area.str.title()
declarations_short_unique.loc[declarations_short_unique.da_area == 'М.Київ', 'da_area'] = 'Київ'
declarations_short_unique['division_address'] = declarations_short_unique.division_address.str.title()
declarations_short_unique['division_address'] = declarations_short_unique.division_address.str[0].str.lower() + declarations_short_unique.division_address.str[1:]
declarations_short_unique['division_address'] = declarations_short_unique.division_address.str.replace('Будинок', 'будинок')


In [9]:
repl = lambda m: m.group(0).lower()
declarations_short_unique['division_address'] = (declarations_short_unique.division_address
                                                    .str.replace("((?<=').)", repl))

In [10]:
(declarations_short_unique.drop('le_name', axis=1)
    .to_json('doctors_for_table.json', force_ascii=False, orient='records'))

In [119]:
to_replace = {"Кам'Янський": "Кам'янський", "Кам'Янка": "Кам'янка","Об'Єднаної": "Об'єднаної",
              "П'Ятий": "П'ятий", "Дем'Янівна": "Дем'янівна"}

In [120]:
hospital_out['le_name'] = hospital_out.le_name.replace(to_replace, regex=True).replace("''", "").str.strip()

In [121]:
hospital_out = (declarations_short_unique[['le_transfer', 'le_name']]
    .drop_duplicates(['le_transfer', 'le_name'])
    .reset_index(drop=True))


hospital_out['le_name'] = (hospital_out
    .le_name.str.replace('Комунальне Некомерційне Підприємство', '')
    .str.replace("Комунальне Медичне Підприємство", "")
    .str.replace("Приватне Акціонерне Товариство", '')
    .str.replace("Центр Первинної Медичної Допомоги", '')
    .str.replace("Товариство З Обмеженою Відповідальністю", ""))


In [122]:
hospital_out.to_json('hospital_names.json', force_ascii=False, orient='records')